** LSTM - Single alphabet model**

In this , we have created LSTM model with single alphabet as parameter. It uses short memory / forgets 
gates to retain patterns learned in sequence useful for predicting target variables. These kinds 
of networks are specialized in learning long term dependencies.They can 
remember the information for a long period of time. LSTM model is useful to predict the names 
which are not in the vocabulary as well. LSTM has this chain-like structure, but instead of 
having a single neural network layer, it has 4 layers interacting with each other In a very special 
way. LSTM take one full vector as input and pass-through all these layers doing a various task 
as addition, concatenation and copying. 



In [ ]:
# Import libraries necessary for this project
from __future__ import print_function
from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
import pandas as pd
import numpy as np
import os
import re

In [ ]:
# determining max lenth for charaters in name
# after 30 it will cut the name
# determing total number of lables
maxlen = 30
labels = 5

# Data Processing 
Here We import the ethnicity data from the file and select the relevant columns. We converted labels into numeric and clean the names by removing suffixes and special characters. Cleaning data is essential as it can create wrong models.

In [ ]:
#reading data and info
df = pd.read_csv("ethnicity collection data11-asian specific.csv")
df.shape
df.head()
df.groupby('ethinicity')['full name'].size()
df =df[['full name','ethinicity']]

# convert the columns into lower case and remove na
# converting lables into numbers
df['full name'] = df['full name'].str.lower()
df['ethinicity']=df['ethinicity'].str.lower()
df['ethinicity']=df['ethinicity'].map({'asian-indian':1,'black non hispanic':2,'hispanic':3 , 'white non hispanic':4,'asian-east':5})
df = df.dropna()

# clean text, remove special charaters, numbers,suffix(Full name),  
def cleaning(text):
    only_words = re.sub('([^A-Za-z ]+)|(^dr\.)|(^dr )|(^mr\.)|(^mr )|(^prof\.)|(^adv\. )',' ',text )
    return only_words

df['full_name_cleaned']=df['full name'].apply(cleaning)
df.groupby('ethinicity')['full name'].size()


# saving data to new file
df.to_csv('data.csv', encoding='utf-8', index=False)
#load new data
cleaned_data = pd.read_csv("data.csv", encoding = "ISO-8859-1")
pd.set_option('display.max_colwidth', None)
#convert lables 
data = cleaned_data.full_name_cleaned
target = cleaned_data.ethinicity  

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2,5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# creating a vocab list 
vocab = set(' '.join([str(i) for i in data]))
vocab.add('END')
vocab.add('-')
len_vocab = len(vocab)

In [ ]:
print(vocab)
print("vocab length is ",len_vocab)
print ("length of input is ",len(cleaned_data))

{'w', 'c', 'l', 'a', 'END', 'y', 't', 'd', '-', 'r', 'i', 'u', 'h', 'e', 'm', 'q', 'p', 'z', 'b', 'n', 'o', ' ', 'k', 'v', 'f', 'x', 'j', 's', 'g'}
vocab length is  29
length of input is  623227


In [ ]:
# converting vocab into dictionary
char_index = dict((c, i) for i, c in enumerate(vocab))
print(char_index)

{'w': 0, 'c': 1, 'l': 2, 'a': 3, 'END': 4, 'y': 5, 't': 6, 'd': 7, '-': 8, 'r': 9, 'i': 10, 'u': 11, 'h': 12, 'e': 13, 'm': 14, 'q': 15, 'p': 16, 'z': 17, 'b': 18, 'n': 19, 'o': 20, ' ': 21, 'k': 22, 'v': 23, 'f': 24, 'x': 25, 'j': 26, 's': 27, 'g': 28}


In [ ]:
#train test split
msk = np.random.rand(len(cleaned_data)) < 0.8
train = cleaned_data[msk]
test = cleaned_data[~msk]

In [ ]:
def set_flag(i):
    tmp = np.zeros(29);
    tmp[i] = 1
    return(tmp)

In [ ]:
set_flag(3)

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
#encode to vector space(one hot encoding)
#padd 'END' to shorter sequences
#also convert each index to one-hot encoding
train_X = []
train_Y = []
trunc_train_name = [str(i)[0:maxlen] for i in train['full_name_cleaned']]
for i in trunc_train_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    train_X.append(tmp)
#converting test data labale into vecotrs mannualy.
for i in train['ethinicity']:
    if i == 1:
        train_Y.append([1,0,0,0,0])
    elif i == 2 :
        train_Y.append([0,1,0,0,0])
    elif i == 3 :
        train_Y.append([0,0,1,0,0])
    elif i == 4 : 
        train_Y.append([0,0,0,1,0])
    elif i == 5 :
        train_Y.append([0,0,0,0,1])


#Model creation
The first step in LSTM is to decide which information is not useful and throw away 
from the cell state. The decision is made by the sigmoid layer , it looks at the input (𝐻𝑡 −
1 𝑎𝑛𝑑 𝑋𝑡) and assign a number between 0 and 1 as a output to cell state. In this case, 0 indicates
that cell needs to be completely forgotten this while 1 represent completely keep this.Once this 
20
is done the vector is pass to the second layer where what new information to be stored in cell 
is decided .This part is divided into two part:

1) A layer to decided which values to be updated

2)A 𝑡𝑎ℎ layer, which creates a new vector for new candidate values(𝐶𝑡). 

In the end, both were combined to update the state of the cell (𝐶𝑡). In this step, the model does 
what it has decided to do in the first step. It forgets what It has decided to forget and update the 
new state of the cell by the scale decided. In the end, the model decides the output it is going 
to generate. The output is based on the cell state, which was updated in the previous step. The 
sigmoid layer is again used here to decide what part of the cell state will be output. After this,
the cell state passes through and multiply it by the output of the sigmoid gate. With our text 
classification problem, we input a name as a sequence of alphabets or bi grams related to each 
other in some way. We make predictions at the end of the article when the model is fed with 
all the name's alphabets. The input is the sequence of the alphabet or bi grams, and output is 
one label. 

For the purposes of this project, the following steps have been used for our model :


1.   We used a stacked LSTM model with 2 LSTM layers for only the characters model
2.    We used the final dense layer with SoftMax activation
3. Used many to one architecture
4. Cross – entropy loss is used with "Adam" Optimizer
5. To avoid the problem of overfitting, we also used 0.2 dropouts and 0.2 recurrent dropouts
6. Model was fir with 10 epochs with a batch size of 100. 


In [ ]:
#build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(maxlen,len_vocab)))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

print(model.summary())

Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 128)           80896     
_________________________________________________________________
dropout (Dropout)            (None, 30, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 5)                 645       
_________________________________________________________________
activation (Activation)      (None, 5)                 0         
Total params: 213,125
Trainable params: 213,125
Non-trainable params: 0
___________________________________

In [ ]:
#creating test data
test_X = []
test_Y = []
trunc_test_name = [str(i)[0:maxlen] for i in test['full_name_cleaned']]
for i in trunc_test_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    test_X.append(tmp)
#converting test data labale into vecotrs mannualy.
for i in test['ethinicity']:
  if i == 1.0:
    test_Y.append([1,0,0,0,0])
  elif i == 2.0:
    test_Y.append([0,1,0,0,0])
  elif i == 3.0:
    test_Y.append([0,0,1,0,0])
  elif i == 4.0: 
    test_Y.append([0,0,0,1,0])
  elif i == 5.0:
    test_Y.append([0,0,0,0,1])


In [ ]:
print(np.asarray(test_X).shape)
print(np.asarray(test_Y).shape)
test_X = np.array(test_X)
test_Y =np.array(test_Y)
train_X = np.array(train_X)
train_Y = np.array(train_Y)

(124753, 30, 29)
(124753, 5)


In [ ]:
#model fitting
batch_size=100
model.fit(train_X, train_Y,batch_size=batch_size,epochs=10,validation_data=(test_X, test_Y))

Epoch 1/10
4985/4985 [==============================] - 882s 177ms/step - loss: 0.4883 - accuracy: 0.8452 - val_loss: 0.4061 - val_accuracy: 0.8713
Epoch 2/10
4985/4985 [==============================] - 875s 176ms/step - loss: 0.3822 - accuracy: 0.8783 - val_loss: 0.3505 - val_accuracy: 0.8884
Epoch 3/10
4985/4985 [==============================] - 871s 175ms/step - loss: 0.3485 - accuracy: 0.8890 - val_loss: 0.3284 - val_accuracy: 0.8944
Epoch 4/10
4985/4985 [==============================] - 870s 175ms/step - loss: 0.3273 - accuracy: 0.8955 - val_loss: 0.3155 - val_accuracy: 0.8984
Epoch 5/10
4985/4985 [==============================] - 876s 176ms/step - loss: 0.3120 - accuracy: 0.9002 - val_loss: 0.3083 - val_accuracy: 0.9009
Epoch 6/10
4985/4985 [==============================] - 872s 175ms/step - loss: 0.3001 - accuracy: 0.9042 - val_loss: 0.3034 - val_accuracy: 0.9025
Epoch 7/10
4985/4985 [==============================] - 871s 175ms/step - loss: 0.2900 - accuracy: 0.9067 - val_

In [ ]:
#model score
score, acc = model.evaluate(test_X, test_Y)
print('Test score:', score)
print('Test accuracy:', acc)

3899/3899 [==============================] - 76s 20ms/step - loss: 0.2927 - accuracy: 0.9059
Test score: 0.29269641637802124
Test accuracy: 0.9059421420097351


In [ ]:

evals = model.predict_classes(test_X)
prob_m = [i[0] for i in evals]

In [ ]:
y_pred = model.predict_classes(test_X, verbose=2)
p = model.predict_proba(test_X, verbose=2) # to predict probability

In [ ]:
target_names= ['asian-indian','black non hispanic','hispanic' , 'white non hispanic','asian-east']

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
print(classification_report(np.argmax(test_Y, axis=1), y_pred, target_names=target_names))
print(confusion_matrix(np.argmax(test_Y, axis=1), y_pred))

                    precision    recall  f1-score   support

      asian-indian       0.98      0.98      0.98     19174
black non hispanic       0.66      0.39      0.49      8337
          hispanic       0.80      0.77      0.79     10861
white non hispanic       0.92      0.96      0.94     83334
        asian-east       0.91      0.87      0.89      3047

          accuracy                           0.91    124753
         macro avg       0.85      0.79      0.82    124753
      weighted avg       0.90      0.91      0.90    124753

[[18807    44    62   217    44]
 [   61  3251   215  4798    12]
 [   97   202  8340  2112   110]
 [  211  1393  1668 79979    83]
 [   72    34    93   206  2642]]


All the LSTM model works equally on the dataset. When we 
used bi -gram model on our data, the precision and recall score for the full name model was 
0.83 and 0.84. In comparison, single alphabet model was 0.90 and 0.90, respectively. When models were 
compared, we observed that the average recall score of all groups except black non-Hispanic 
is high. This tells us that , Black non-Hispanic people are challenging to predict. The cause of this might be less data for 
that class or not specific pattern in names. However, here we focus on the 'Asian- Indian 
Subcontinent" group. We realized that score for this specific ethnicity was between 0.96 -0.98 
across three models.Choosing 
the final model was difficult. We decided to go with LSTM – Single alphabet model causes the 
accuracy and other ethnic groups' recall scores better than other models

In [ ]:
model.save('singlealphabet_lstm.h5')

In [ ]:
import csv
with open('singlealphabet_lstm_vocab.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in char_index.items():
       writer.writerow([key, value])

#Deploying model

Deploying the model in real-world data is the final part of the project. Deploying requires cleaning the data, importing vocabulary and models, and processing the data.

In [ ]:
# LSTM SINGLE ALPHABET MODEL DEPOLYMENT

import re, nltk
import numpy as np
import pandas as pd
import csv
from keras.models import load_model

#loading compnay data , info and selecting valid column
df = pd.read_csv("county.csv")
df.shape
df.head()
df =df[['prop_owner_name', 'IsSelectedforExport']]

# convert the columns into lower case and remove na
df['prop_owner_name'] = df['prop_owner_name'].str.lower()
df = df.dropna()

#cleaning unwanted data
df = df[~(df.prop_owner_name.str.contains('llc'))]
df = df[~(df.prop_owner_name.str.contains('ltd'))]
df = df[~(df.prop_owner_name.str.contains('estate'))]
df = df[~(df.prop_owner_name.str.contains('trust'))]
df = df[~(df.prop_owner_name.str.contains('inc'))]
df = df[~(df.prop_owner_name.str.contains('trustee'))]

new=df['prop_owner_name'].str.split("&",n = 1, expand = True)
df['name1']= new[0]

# clean text, remove special charaters, numbers,suffix(Full name),  
def cleaning(text):
    only_words = re.sub('([^A-Za-z ]+)|(^dr.)|(^dr )|(^mr.)|(^mr )|(^prof.)|(^adv. )',' ',text )
    return only_words
df['name1']=df['name1'].apply(cleaning)

# loading model
model = load_model('singlealphabet_lstm.h5')
vocab = pd.read_csv("singlealphabet_lstm_vocab.csv")

# converting vocab into dictonary
char_index= {}
for i, word in enumerate(vocab['vacab']):
         char_index[word] = i
# setting vector size to vocab size
def set_flag(i):
    tmp = np.zeros(29);
    tmp[i] = 1
    return(tmp)

#converting names
name=df['name1'].to_list()
maxlen = 30
labels = 5
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)

# predicitng using model
pred=model.predict(np.asarray(X))
len(pred)

# giving lables to predicited values
labels = ['asian-indian','black non hispanic','hispanic', 'white non hispanic','asian-east']
a = list(pred)

# appending predicited lables to list
b = []
for i in a :
    l =labels[np.argmax(i)]
    b.append(l)
 
# saving predicited outcome
df['prediciton']= b
df.to_csv('lstm_predicted_ singleaphabet.csv', encoding='utf-8', index=False)